In [ ]:
!pip install spotipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.0/252.0 kB 4.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import csv
from collections import defaultdict
import math
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive
from sklearn.cluster import KMeans
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
import requests
import re
import sys
import random
import spotipy
from spotipy.oauth2 import SpotifyOAuth
from spotipy.oauth2 import SpotifyClientCredentials
%matplotlib inline
drive.mount("/content/drive", force_remount = True)


ValueError: mount failed

In [ ]:
basePath = "/content/drive/MyDrive/AI_Project/"

In [ ]:
data = basePath + "genre_attrs.csv"

In [ ]:
# Loading user's playlist data, extracts all the genres, puts genres into a dictionary


# Initialize a defaultdict to store genres and their frequencies
genre_frequency = defaultdict(int)

# Path to your CSV file
file_path = basePath + 'past_lives.csv'

# Reading and parsing the CSV file
with open(file_path, newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)

    # Iterate through each row in the CSV
    for row in reader:
        genres = row['Genres'].split(',')  # Split genres based on comma

        # Iterate through each genre in the 'Genres' column
        for genre in genres:
            # Remove leading/trailing spaces and convert to lowercase for uniformity
            cleaned_genre = genre.strip().lower()

            # Increment the count for the genre in the defaultdict
            genre_frequency[cleaned_genre] += 1

# Displaying the hashmap with genres and their frequencies
for genre, frequency in genre_frequency.items():
    print(f"{genre}: {frequency}")

In [ ]:
data_pd = pd.read_csv(data, sep = ',', engine = 'python')

# data_pd(by=['hex_colour'])
print(data_pd)

In [ ]:
# Selecting features for clustering
features = data_pd[['x', 'y']]


# Scaling data
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)

# Applying k-means clustering
k = 50
kmeans = KMeans(n_clusters=k, init='k-means++', random_state=42)
data_pd['cluster'] = kmeans.fit_predict(scaled_features)
labels = kmeans.labels_
centroids_scaled = kmeans.cluster_centers_

centroids = scaler.inverse_transform(centroids_scaled)

# Visualizing clusters and centroids using seaborn
plt.figure(figsize=(15, 15))
sns.scatterplot(x='x', y='y', hue='cluster', data=data_pd, palette='tab10', alpha=0.5)
plt.scatter(centroids[:, 0], centroids[:, 1], s=200, c='red', marker = 'X', label = 'Centroids')  # Plotting centroids
plt.xlabel('x')
plt.ylabel('y')
plt.title('K-means Clustering')
plt.show()

In [ ]:
x_list = []
y_list = []
g_list = []
c_list = []


for q in range(0, 5453):
    x_list.append(data_pd.loc[q].at["x"])
    y_list.append(data_pd.loc[q].at["y"])
    c_list.append(data_pd.loc[q].at["hex_colour"])
    g_list.append(data_pd.loc[q].at["genre"])


print(x_list)
print(y_list)
print(g_list)
plt.figure(figsize=(15, 15))
plt.scatter(x_list ,y_list, c = c_list)
plt.xlabel('X Axis')
plt.ylabel('Y Axis')
plt.title('EveryNoise Scatter Plot')
plt.show()

In [ ]:
coordinates = list(zip(x_list, y_list))
#print(coordinates)

search_genre = input("Enter a Genre: ").lower()
#print(search_genre)

cluster_base = data_pd.loc[data_pd["genre"].str.lower() == search_genre]
print(cluster_base)

print(cluster_base['cluster'].values[0])

if not cluster_base.empty:
    search_x = cluster_base.iloc[0].at["x"]
    search_y = cluster_base.iloc[0].at["y"]
    #print(search_x, ",", search_y)

    search_tuple = (search_x, search_y)
    #print(search_tuple)

    try:
        index = coordinates.index(search_tuple)
        print("Index of search_tuple:", index)

        scaling_factor = 100  # Adjust as needed
        scaled_centroids = kmeans.cluster_centers_ * scaling_factor

        # Visualize the clusters and centroids using seaborn
        plt.figure(figsize=(15, 15))
        sns.scatterplot(x='x', y='y', hue='cluster', data=data_pd, palette='tab10', alpha=0.5)
        plt.scatter(search_x, search_y, s=500, c='red', marker='X')
        plt.xlabel('x')
        plt.ylabel('y')
        plt.title('K-means Clustering')
        plt.show()


    except ValueError:
        print("Index not found in coordinates list.")
else:
    print("Genre not found in the dataset.")

In [ ]:
# To find playlist of songs for one genre, need to find a specific webpage, then find a specific playlist link on that webpage. This is the code block for that.
# With the playlist link of songs for one genre, we can have a recommendation algorithm that finds a random song

genre = 'rock'
# genre = 'midwestemo'
# URL of the webpage that has link to genre-specific playlist
url = 'https://everynoise.com/engenremap-' + genre + '.html'

response = requests.get(url) # Fetch the HTML content from the URL


if response.status_code == 200: # Check if the request was successful (status code 200)
    # Extract the HTML content
    html_content = response.text

    # Define the regex pattern to find the specific line
    regex_pattern = r'https://open.spotify.com/playlist/([a-zA-Z0-9]+)'

    # Use regex to find the line containing the link
    match = re.search(regex_pattern, html_content)

    if match:
        spotify_link = 'https://open.spotify.com/playlist/'
        spotify_link += match.group(1)  # Extract the link
        print(f"Found Spotify playlist link: {spotify_link}")
    else:
        print("Link to spotify playlist invalid")
else:
    print('Failed to fetch HTML content from URL')

In [ ]:
CLIENT_ID = '510fecc410c74c528a269892efa46631'
CLIENT_SECRET = 'b6b11204e6e94160842d4b05fce1b158'

In [ ]:
offset = 0
track_ids = []

sp_client = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET))

response = sp_client.playlist_items(spotify_link,
                                offset=offset,
                                fields='items.track.id,total',
                                additional_types=['track'])

for item in response['items']:
    track_id = item['track']['id']
    track_ids.append(track_id)

rand_selection = random.randint(0, len(track_id))

track_info = sp_client.track(track_ids[rand_selection])
track_name = track_info['name']

track_name += " by "
artists = [artist['name'] for artist in track_info['artists']]
for artist in artists:
    track_name += artist + " "

print(track_name)

# offset = offset + len(response['items'])
# print(offset, "/", response['total'])